In [ ]:
import re
import ast
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [174]:
df = pd.read_csv('patchnotes.txt', sep='|', names=['patch', 'date', 'buffs', 'nerfs'])
df.fillna(' ', inplace=True)
df.head(88)

,patch,date,buffs,nerfs
0,7.10,1.484079e+09,"buffs:draven,kalista,lucian,talon","nerfs::camille,karma,kennen,leesin,poppy,ryze,..."
1,5.15,1.438727e+09,buffs::teemo;,"nerfs::nidalee,sivir;"
2,5.16,1.440021e+09,"buffs:azir,kassadin,leesin,leblanc,lissandra,l...","nerfs::elise,kayle,rumble,viktor;"
3,5.17,1.441184e+09,"buffs:bard,darius,draven,jax,karma,lux,nocturn...","nerfs::azir,gangplank,garen,gragas,janna,kalis..."
4,5.18,1.442372e+09,"buffs:anivia,caitlyn,ezreal,jarvaniv,morgana,s...","nerfs:braum,darius,elise,fiora,galio,garen,gan..."
5,5.19,1.443567e+09,"buffs:missfortune,shen,zac",nerfs:devourer
6,5.20,1.444794e+09,"buffs::azir,dr.mundo,teemo","nerfs::darius,evelynn,fiora,vayne"
7,5.21,1.446072e+09,"buffs:chogath,kassadin","nerfs::gangplank,mordekaiser,riven,veigar"
8,5.22,1.447232e+09,"buffs::draven,ezreal,jinx,leblanc,shaco,","nerfs::sivir,kindred,"
9,5.23,1.448298e+09,"buffs:azir,bard,caitlyn,khazix,skarner,zed,zilean","nerfs:lucian,missfortune,"


In [180]:
dfp = pd.read_csv('patchnotes.txt', sep='|', names=['patch', 'date', 'buffs', 'nerfs'])
dfp.fillna(' ', inplace=True)
dfp = dfp[dfp['buffs'].str.contains('buffs')]
dfp['buffs'] = dfp['buffs'].str.replace('buffs:', '')
dfp['buffs'] = dfp['buffs'].str.replace('[:;.]+', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('nerfs:', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('[:.;]+', '')
dfp.sort_values('date', inplace=True)
dfp.reset_index(drop=True, inplace = True)
dfp.head()

,patch,date,buffs,nerfs
0,1.0,1.355431e+09,"chogath,diana,rengar,teemo,zyra,brutalizer,bla...","elise,fiora,masteryi"
1,2.0,1.358293e+09,releasedyet,"amumu,blitzcrank,evelynn,ezreal,katarina,khazi..."
2,3.0,1.359660e+09,"blitzcrank,chogath,ezreal,katarina,missfortune...","heimerdinger,kennen,nasus,sion,tristana,trynda..."
3,3.0,1.360786e+09,,"jarvaniv,lulu"
4,3.5,1.365538e+09,"corki,karma,morgana,nami,nautilus,quinn","akali,kayle,lux,volibear"


In [ ]:
# Patch
response.xpath('//h1[@class="post-title entry-title"]/text()').extract()

# Date
response.xpath('//span[@class="post-timestamp"]/text()').extract()
    
# Buff/nerf/tweaks
response.xpath('//div[@class="post-body entry-content"]//b//text()').extract()

# Changed Champs
#response.xpath('//div[@class="post-body entry-content"]/text()').extract()
#response.xpath('//div[@class="post-body entry-content"]/div[2]//text()').extract()
response.xpath('//div[@class="post-body entry-content"]//b//following-sibling::text()').extract()

In [73]:
from datetime import datetime
import time
import re

patch = ['\nOfficial Patch 6.24 Notes Released!\n']
patch = re.findall(r"[-+]?\d*\.\d+|\d+", patch[0])
print(patch[0])

date = ['\nDec 4, 2016\n9:55 AM\n']
dt = datetime.strptime(date[0].replace('\n', ' '), ' %b %d, %Y %I:%M %p ')
release = time.mktime(dt.timetuple())
print(release)
datetime.fromtimestamp(release).ctime()

champs = [" Akali, Azir, Fiddlesticks, Garen, Katarina, Kog'Maw, Nidalee, Varus",
 '\n',
 ' Ivern, Leblanc, Rengar, Twitch, Vayne',
 '\n',
 ' Master Yi',
 '\n',
 ": Rylai's Crystal Scepter, Liandry's Torment",
 '\n',
 ': Courage of the Colossus, Fervor of Battle',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n']

while '\n' in champs: champs.remove('\n')

champs = [x.replace(' ', '').replace("'", '').replace("'", '').lower() for x in champs]
print(champs)


changes = ['Buffs:', 'Nerfs:', 'Tweaks:', 'Item Changes', 'Mastery Changes']
print(changes)

if any("buffs" in s.lower() for s in changes): print('buffs')

6.24
1480845300.0
['akali,azir,fiddlesticks,garen,katarina,kogmaw,nidalee,varus', 'ivern,leblanc,rengar,twitch,vayne', 'masteryi', ':rylaiscrystalscepter,liandrystorment', ':courageofthecolossus,fervorofbattle']
['Buffs:', 'Nerfs:', 'Tweaks:', 'Item Changes', 'Mastery Changes']
buffs


In [ ]:
starturl = 'www.nerfplz.com/search/label/Patch Notes?max-results=50'
#patch links
response.xpath('//span[@class="rmlink"]/a[contains(text(), "Read More...")]/@href').extract()

#next page
response.xpath('//span[@id="blog-pager-older-link"]/a/@href').extract()